# Write Slurm Files

Write slurm files for simulations varying the chemical potential of the system.

#### USER INPUTS

Specify what chemical potentials to simulate.

In [1]:
all_cp = [i * 0.1 + -10 for i in range(21)]
print(f"Chemical potentials: {all_cp}")

Chemical potentials: [-10.0, -9.9, -9.8, -9.7, -9.6, -9.5, -9.4, -9.3, -9.2, -9.1, -9.0, -8.9, -8.8, -8.7, -8.6, -8.5, -8.4, -8.3, -8.2, -8.1, -8.0]


Specify the number of replicates to run.

In [2]:
n_replicates = 20

Specify the resources to request for each job.

In [3]:
runtime = "2:59:59"
num_tasks = "1"
cpus_per_task = "1"
mem_per_cpu = "4G"

#### Import Modules

In [4]:
import os
import sys
from datetime import datetime
import numpy as np

#### Navigate to Root Directory

In [5]:
# Get the absolute path of the notebook's directory
notebook_directory = os.path.dirname(os.path.abspath('__file__'))

# Navigate one level up to get the root directory path
root_directory = os.path.abspath(os.path.join(notebook_directory, '..'))

# Change the working directory to the root directory
os.chdir(root_directory)

# Append the root directory to sys.path
sys.path.append(root_directory)

#### Generate Directory for SBATCH files

In [6]:
# Generate a directory for ALL sets of sbatch files (if it does not exist)
sbatch_dir = "sbatch_files"
if not os.path.exists(sbatch_dir):
    os.makedirs(sbatch_dir)

In [7]:
# Generate a subdirectory for the CURRENT set of sbatch files
now = datetime.now()
subdirectory_name = now.strftime("var_cp_%Y-%m-%d_%H-%M-%S")
slurm_dir = os.path.join(sbatch_dir, subdirectory_name)
os.makedirs(slurm_dir)

In [8]:
analysis_dir = "/scratch/users/jwakim/sliding_nucleosome/examples"
analysis_file = "vary_cp.py"
slurm_prefix = "vary_cp"
run_file_prefix = "run_jobs"
job_prefix = "cp"

In [9]:
job_count = 0
batch_size = 900
n_batches = int(np.ceil(len(all_cp) * n_replicates / batch_size))
run_files = {i: [] for i in range(n_batches)}

for i, cp in enumerate(all_cp):
    for _ in range(n_replicates):
        batch_ind = int(job_count // batch_size)
        slurm_file_name = f"{slurm_prefix}_{job_count}.slurm"
        job_name = f"{job_prefix}_{job_count}"
        run_files[batch_ind].append(slurm_file_name)
        slurm_file_path = os.path.join(slurm_dir, slurm_file_name)

        with open(slurm_file_path, 'w') as f:

            f.write("#!/bin/bash\n\n")

            f.write(f"#SBATCH --job-name={job_name}\n")
            f.write(f"#SBATCH --time={runtime}\n")
            f.write(f"#SBATCH --ntasks={num_tasks}\n")
            f.write(f"#SBATCH --cpus-per-task={cpus_per_task}\n")
            f.write(f"#SBATCH --mem-per-cpu={mem_per_cpu}\n\n")

            f.write(f"source ~/.bashrc\n")
            f.write(f"conda activate slide\n")
            f.write(f"cd {analysis_dir}\n")
            f.write(f"python {analysis_file} {cp}\n")
            f.write(f"echo 'Job complete!'\n")

        job_count += 1

In [10]:
for i in range(n_batches):
    run_file_name = f"{run_file_prefix}_{i}.sh"
    with open(os.path.join(slurm_dir, run_file_name), "w") as f:
        for job_file in run_files[i]:
            f.write(f"sbatch {job_file}\n")